このnotebookではkerasの基本的な使い方を学ぶために, mnistの手書き文字の分類をおこないます。

まず訓練画像と対応する訓練ラベル、テスト画像と対応するテストラベルを読み込みます。
ここで読み込まれたデータのtypeはnumpy.ndarrayになっています。

In [0]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print('train_images.shape', train_images.shape)
print('train_labels.shape', train_labels.shape)
print('test_images.shape', test_images.shape)
print('test_labels.shape', test_labels.shape)

データの前処理を行います。

データの形状が今は(28, 28)の二次元配列になっているため、(28\*28)の一次元配列にします。
また、現在は[0, 255]の画素値が含まれていますが、これを0.0 ~ 1.0の値に正規化します。
さらに、データのタイプを必ずfloat32型にしてください。

In [0]:
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype('float32') / 255.

test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32') / 255.

また、ラベルを整数からOne-hot vectorにエンコードします。

(例) 5 -> [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [0]:
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

次に、ニューラルネットワークを構築しましょう。
今回のネットワークは二層の全結合層(layers.Dense)からなるネットワークになります。

In [0]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28*28,)))
network.add(layers.Dense(10, activation='softmax'))

つぎに、生成したネットワークをコンパイルします。ネットワークのコンパイルには次の三つの要素を決める必要があります。
- optimizer (最適化アルゴリズム)

  SGDなどの最適化アルゴリズムとなります。今回はRMSpropというアルゴリズムにしてみましょう。
- loss (損失関数)

  小さくするべき損失関数です。今回はクロスエントロピー誤差を用いてみましょう。
- metrics

  ネットワークの性能を表す評価関数です。損失関数と異なり、ネットワークの学習には使われません。今回は正解率(正しく分類できた割合)にします。

In [0]:
network.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy']
               )

ネットワークを訓練するために、fitメソッドを呼び出します。

In [0]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

最後に訓練したモデルをテストデータに適用してみましょう。

In [0]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc', test_acc)

正しく学習できている場合、訓練データの正解率が99%弱、テストデータの正解率が98%弱になっていると思います。

第一回のハンズオンで組み立てたコードと似たものが、20行前後でかけていることがわかります。このように、kerasなどのフレームワークを用いることで複雑なモデルも簡単に記述することができます。